In [ ]:
import sys
from pathlib import Path
cwd = Path.cwd()
if (cwd / 'src').exists():
    ROOT = cwd
else:
    ROOT = cwd.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
if str(ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(ROOT / 'src'))

import importlib
importlib.invalidate_caches()

import numpy as np
from src.analysis.identifiability import profile_likelihood, fit_wrapper_for_profile_local

# Example: make toy data and run a quick profile
from src.models.leloup_goldbeter import LGParams, default_initial_conditions
from src.estimation.estimation import theta_to_params, fit_local

p0 = LGParams()
t_obs = np.linspace(0, 48, 40)
y0 = default_initial_conditions()
from src.estimation.estimation import simulate_model
X = simulate_model(p0, y0, t_obs)

# Use 4 mRNA observational indices
observed_states = [0, 3, 6, 9]
y_obs = X[observed_states]

# initial vector (8 params used by theta_to_params)
theta0 = np.array([p0.v_sP, p0.v_sC, p0.v_sB, p0.v_sR,
                   p0.k_degM_P, p0.k_degM_C, p0.k_degM_B, p0.k_degM_R])

# profile over first param
param_grid = np.linspace(theta0[0]*0.5, theta0[0]*1.5, 15)
pg, obj = profile_likelihood(0, param_grid, theta0, fit_wrapper_for_profile_local, (t_obs, y_obs))

print('Profile grid', pg)
print('Objective values', obj)